In [10]:
%%capture
!pip install praw tweepy pandas requests beautifulsoup4 python-dotenv textblob tqdm

In [1]:
import praw
import tweepy
import json
import os
import time
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from textblob import TextBlob
from tqdm import tqdm
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 0  # To make detection deterministic

### Fetch Credentials

In [2]:
load_dotenv()

# Retrieve credentials
reddit_client_id = os.getenv("REDDIT_CLIENT_ID")
reddit_client_secret = os.getenv("REDDIT_CLIENT_SECRET")
reddit_user_agent = os.getenv("REDDIT_USER_AGENT")

### Crawling Configurations

In [3]:
# Crawler default settings
DEFAULT_REDDIT_LIMIT = 500

# Subreddits to crawl
SUBREDDITS = [
    'CryptoCurrency',
    'CryptoMarkets',
    'binance',
    'CoinBase',
    'Crypto_com',
    'kucoin',
    'BitcoinBeginners',
    'CryptoScams',
    'Kraken',
    'Bybit',
    'OKX',
    'CryptoTechnology',
    'Ethereum'
]

# Output file paths
REDDIT_OUTPUT_CSV = "../data/reddit_crypto_data.csv"

# Crypto Exchanges for detection
CRYPTO_EXCHANGES = {
    'binance': ['binance', 'bnb', 'binance us', 'binance app', 'binance exchange'],
    'coinbase': ['coinbase', 'coinbase pro', 'coinbase wallet', 'cb wallet'],
    'kraken': ['kraken', 'kraken exchange', 'kraken pro'],
    'okx': ['okx', 'okex'],
    'kucoin': ['kucoin', 'kucoin exchange'],
    'crypto.com': ['crypto.com', 'crypto.com app'],
    'bybit': ['bybit', 'bybit app']
}

### Sentiment Analysis Method
- We want a balanced number of positive, negative and neutral

In [4]:
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

### Reddit Crawler Setup

In [5]:
class RedditCrawler:
    """Class for crawling Reddit posts and comments related to crypto exchanges."""

    def __init__(self, client_id, client_secret, user_agent):
        self.reddit = praw.Reddit(
            client_id=client_id,
            client_secret=client_secret,
            user_agent=user_agent
        )
        self.data = []

    def crawl_subreddit(self, subreddit_name, limit=2000, search_query=None):
        subreddit = self.reddit.subreddit(subreddit_name)
        submissions = subreddit.search(search_query, limit=limit) if search_query else subreddit.top(time_filter="all", limit=limit)

        for submission in tqdm(submissions, desc=f"Crawling r/{subreddit_name}"):
            post_text = f"{submission.title} {submission.selftext}".strip()
            platform = self._detect_crypto_exchange(post_text)

            if not self._is_valid_english_opinion(post_text, platform):
                continue

            post_data = {
                'id': submission.id,
                'title': submission.title,
                'text': submission.selftext,
                'score': submission.score,
                'created_utc': datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'author': str(submission.author),
                'num_comments': submission.num_comments,
                'subreddit': subreddit_name,
                'permalink': f"https://reddit.com{submission.permalink}",
                'type': 'submission',
                'platform': platform,
                'sentiment': get_sentiment(post_text)
            }
            self.data.append(post_data)

            # Process comments
            submission.comments.replace_more(limit=5)
            for comment in submission.comments.list():
                comment_text = comment.body.strip()
                platform = self._detect_crypto_exchange(comment_text)

                if not self._is_valid_english_opinion(comment_text, platform):
                    continue

                comment_data = {
                    'id': comment.id,
                    'text': comment_text,
                    'score': comment.score,
                    'created_utc': datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    'author': str(comment.author),
                    'parent_id': comment.parent_id,
                    'subreddit': subreddit_name,
                    'permalink': f"https://reddit.com{submission.permalink}{comment.id}/",
                    'type': 'comment',
                    'platform': platform,
                    'sentiment': get_sentiment(comment_text)
                }
                self.data.append(comment_data)

    def _detect_crypto_exchange(self, text):
        text = text.lower()
        for exchange, keywords in CRYPTO_EXCHANGES.items():
            for keyword in keywords:
                if keyword in text:
                    return exchange
        return 'general'

    def _is_valid_english_opinion(self, text, platform):
        """Filter out short, non-English, or general-opinion content."""
        if not text or len(text) < 20:
            return False
        if platform == 'general':
            return False
        try:
            return detect(text) == 'en'
        except:
            return False

    def save_to_csv(self, filename):
        df = pd.DataFrame(self.data)
        df.drop_duplicates(subset=['id'], inplace=True)
        df.to_csv(filename, index=False)
        print(f"Saved {len(df)} records to {filename}")
        return df

    def save_to_json(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.data, f)
        print(f"Saved {len(self.data)} records to {filename}")

### Execute Crawling Process

In [6]:
# Initialise Reddit Crawler
reddit_crawler = RedditCrawler(
    client_id=reddit_client_id,
    client_secret=reddit_client_secret,
    user_agent=reddit_user_agent
)

In [7]:
# Crawl Reddit
for subreddit in SUBREDDITS:
    reddit_crawler.crawl_subreddit(subreddit, limit=DEFAULT_REDDIT_LIMIT)

# Save Reddit Data
reddit_df = reddit_crawler.save_to_csv(REDDIT_OUTPUT_CSV)

# Optionally Display Data to Inspect
reddit_df

Crawling r/CryptoCurrency: 112it [01:09,  1.62it/s]


KeyboardInterrupt: 